# Reduced model

## Data Import

In [1]:
# Import libraries
import pandas as pd
import numpy as np
# preprocessing
from sklearn.preprocessing import label_binarize, MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion

# Modeling
from sklearn.naive_bayes import GaussianNB
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import MultiOutputClassifier
# Data visualization
%matplotlib inline
from itertools import cycle
import seaborn as sns

# Performance metrics
from sklearn import metrics
import joblib

In [2]:
data = pd.read_excel('../data/processed/cleaned_data.xlsx', index_col='Scene no.').copy()
print(data.columns)
features_10 = ['evaporation_and_natural_disperson','E_ss', 'E_sl', 'E_sw', 'sufficient_mixing_energy','E_ssC', 'seawater', 'E_ssI','soot_pollution', 'displacement']

X_reduced = data[features_10]
y = data[['mcr_DT_output', 'cdu_DT_output', 'isb_DT_output']]

# Train test Split
X_train, X_test, y_train, y_test = train_test_split(X_reduced, y, test_size=0.20, random_state=12)

Index(['oil_spill_size', 'evaporation_and_natural_disperson', 'persistence',
       'oil_amount_to_recover', 'E_ss', 'E_sl', 'E_sw', 'E_sb',
       'sufficient_mixing_energy', 'seasurface', 'seawater', 'Rtime_ss',
       'Rtime_sw', 'E_ssC', 'E_slC', 'E_swC', 'E_sbC', 'soot_pollution',
       'residue_recovery', 'displacement', 'E_ssI', 'E_slI', 'E_swI', 'E_sbI',
       'shoreline_length', 'distance_to_inhabitation', 'mcr_DT_output',
       'cdu_DT_output', 'isb_DT_output'],
      dtype='object')


In [3]:
# Drop y with Consider class which has only 5 records
#y = y[y['CDU options'] != 8]
#X = X.drop(['Scene 26', 'Scene 1247', 'Scene 1380', 'Scene 1655', 'Scene 2109']) # y[y['CDU options'] == 8].index.values

In [13]:
display(X_reduced)
print(X_test.columns)

,evaporation_and_natural_disperson,E_ss,E_sl,E_sw,sufficient_mixing_energy,E_ssC,seawater,E_ssI,soot_pollution,displacement
Scene no.,,,,,,,,,,
Scene 1,16,0,0,1,no,0,Small,0,YES soot pollution,yes
Scene 2,34,-1,1,-1,yes,-1,Large,0,YES soot pollution,no
Scene 3,74,0,1,0,yes,1,Large,1,NO soot pollution,no
Scene 4,41,0,0,-1,no,0,Large,0,YES soot pollution,yes
Scene 5,55,-1,0,0,yes,0,Large,1,NO soot pollution,no
...,...,...,...,...,...,...,...,...,...,...
Scene 2696,20,-1,1,-1,no,1,Large,1,YES soot pollution,yes
Scene 2697,74,0,0,0,no,-1,Large,-1,NO soot pollution,yes
Scene 2698,52,0,-1,-1,yes,0,Large,0,YES soot pollution,yes


Index(['evaporation_and_natural_disperson', 'E_ss', 'E_sl', 'E_sw',
       'sufficient_mixing_energy', 'E_ssC', 'seawater', 'E_ssI',
       'soot_pollution', 'displacement'],
      dtype='object')


## Modeling
### Training

In [5]:
# modeling
numerical_features = tuple(X_reduced._get_numeric_data().columns)
categorical_features = list(set(X_reduced.columns) - set(numerical_features) - set(['displacement.1', 'oil_spill_size', 'Rtime_ss', 'Rtime_sw']))

numerical_transformer = Pipeline(steps=[("MinMaxScaler", MinMaxScaler(feature_range=(1,2)))]) # numerical_features
categorical_transformer = Pipeline(steps=[("ohe", OneHotEncoder(handle_unknown="ignore",sparse=False))])
# ++ need to add spill size convert 4,699,9999

# TRANSFORM NUMERICAL & CATEGORICAL FEATURES SEPARATELY USING ColumnTransformer
feature_pipeline = ColumnTransformer(transformers=[
            ('num', numerical_transformer, numerical_features),
            ('cat', categorical_transformer, categorical_features)],
         remainder='drop')


steps=[("feature_pipeline", feature_pipeline),
      ("model",MultiOutputClassifier(GaussianNB()))]

pipe=Pipeline(steps)
#model_BIMReTA = MultiOutputClassifier(model_GB_ins).fit(X_train, y_train)
model_BIMReTA = pipe.fit(X_train,y_train)

/Users/tanmoydas/Applications/anaconda3/envs/PhD_venv/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


### Save the reduced model

In [6]:
joblib.dump(model_BIMReTA, '../models/model_BIMReTA.pkl')

['../models/model_BIMReTA.pkl']

### Classification/ Ranking

In [7]:
# Classify
y_pred=model_BIMReTA.predict(X_test) 

print(X_test.shape)
#y_pred = model_BIMReTA.predict(X_test)
y_score = model_BIMReTA.predict_proba(X_test)
print(y_pred)

(540, 10)
[['ok ' 'Not recommended ' 'Unknown']
 ['ok ' 'OK ' 'OK ']
 ['ok ' 'Not recommended ' 'Consider ']
 ...
 ['ok ' 'Not recommended ' 'Unknown']
 ['Unknown' 'OK ' 'Consider ']
 ['ok ' 'Not recommended ' 'OK ']]


In [44]:
X_test.columns

Index(['evaporation_and_natural_disperson', 'E_ss', 'E_sl', 'E_sw',
       'sufficient_mixing_energy', 'E_ssC', 'seawater', 'E_ssI',
       'soot_pollution', 'displacement'],
      dtype='object')

In [50]:
X1 = pd.DataFrame(np.array([[99, 1, 1, 1, 'no', 0, 'Small', 1, 0, 'yes']]))
X1.columns = ['evaporation_and_natural_disperson', 'E_ss', 'E_sl', 'E_sw',
       'sufficient_mixing_energy', 'E_ssC', 'seawater', 'E_ssI',
       'soot_pollution', 'displacement']

y_pred1=model_BIMReTA.predict(X1) 
y_pred1
#mody_pred1el.predict([[dispersion, E_ss, E_sl, E_sw, sufficient_mixing_energy,
#                                 E_ssC, seawater, E_ssI, soot_pollution, displacement]])
#model_BIMReTA.predict(y_pred1)

array([['ok ', 'Not recommended ', 'OK ']], dtype='<U18')

In [53]:
result = y_pred1# rank_response_technologies(dispersion, E_ss, E_sl, E_sw, sufficient_mixing_energy,
          #                          E_ssC, seawater, E_ssI, soot_pollution, displacement)
result_df = pd.DataFrame(columns=["MCR", "CDU", "ISB"])
result_df.index = ['Ranking']
result_df.append(result)
result_df
#df.index = [100, 200, 300]

ValueError: Length mismatch: Expected axis has 0 elements, new values have 1 elements

In [55]:
data = dict(MCR=1, CDU=1.23, ISB='Hello')
df = pd.DataFrame(data, index=['Ranking'])
df

,MCR,CDU,ISB
Ranking,1,1.23,Hello


In [28]:
X_test[[0]]

KeyError: "None of [Index([0], dtype='int64')] are in the [columns]"

### Model Assessment

In [8]:
print('----------------------------Confusion Matrix--------------')
print('MCR')
display(metrics.confusion_matrix(y_test.iloc[:,0], y_pred[:,0]))

print('CDU')
display(metrics.confusion_matrix(y_test.iloc[:,1], y_pred[:,1]))

print('ISB')
display(metrics.confusion_matrix(y_test.iloc[:,2], y_pred[:,2]))


print('----------------------------Classification Report--------------')
print('MCR')
print(metrics.classification_report(y_test.iloc[:,0],y_pred[:,0]))
print('CDU')
print(metrics.classification_report(y_test.iloc[:,1],y_pred[:,1]))
print('ISB')
print(metrics.classification_report(y_test.iloc[:,2],y_pred[:,2]))

----------------------------Confusion Matrix--------------
MCR


array([[ 54,   2,   0,   0],
       [ 25,   0,   0,   0],
       [  0,   0, 112,  89],
       [ 68,   1,  68, 121]])

CDU


array([[  1,   0,   0,   0],
       [  8, 249,  81,   0],
       [ 10,   0, 173,   0],
       [ 18,   0,   0,   0]])

ISB


array([[103,   0,  33],
       [  0, 349,  24],
       [  0,   0,  31]])

----------------------------Classification Report--------------
MCR
                    precision    recall  f1-score   support

         Consider        0.37      0.96      0.53        56
Go to next season        0.00      0.00      0.00        25
           Unknown       0.62      0.56      0.59       201
               ok        0.58      0.47      0.52       258

          accuracy                           0.53       540
         macro avg       0.39      0.50      0.41       540
      weighted avg       0.54      0.53      0.52       540

CDU
                  precision    recall  f1-score   support

       Consider        0.03      1.00      0.05         1
Not recommended        1.00      0.74      0.85       338
             OK        0.68      0.95      0.79       183
         Unknown       0.00      0.00      0.00        18

        accuracy                           0.78       540
       macro avg       0.43      0.67      0.42       540
    weighted avg       0.86      0.78

/Users/tanmoydas/Applications/anaconda3/envs/PhD_venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/tanmoydas/Applications/anaconda3/envs/PhD_venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/tanmoydas/Applications/anaconda3/envs/PhD_venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this 

In [9]:
print('----------------------------ROC AUC--------------')
print('MCR')
print(metrics.roc_auc_score(y_test.iloc[:,0],y_score[0], multi_class='ovo'))
print('CDU')
print(metrics.roc_auc_score(y_test.iloc[:,1],y_score[1], multi_class='ovo'))
print('ISB')
print(metrics.roc_auc_score(y_test.iloc[:,2],y_score[2], multi_class='ovo'))

----------------------------ROC AUC--------------
MCR
0.799523529454977
CDU
0.892749832640543
ISB
0.9703177647431885


In [10]:
y_score[1]

array([[0.00000000e+00, 1.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 7.96371104e-11, 1.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 1.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [0.00000000e+00, 1.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 1.07401390e-10, 1.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 1.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

## Compare with Full model

### Data of full & reduced model

In [11]:
# 
# import data
X = pd.read_excel('Inputs/X.xlsx', index_col='Scene no.').copy()
y = pd.read_excel('Inputs/y.xlsx', index_col='Scene no.').copy()

# Drop y with Consider class which has only 5 records
#y = y[y['CDU options'] != 8]
#X = X.drop(['Scene 26', 'Scene 1247', 'Scene 1380', 'Scene 1655', 'Scene 2109']) # y[y['CDU options'] == 8].index.values



FileNotFoundError: [Errno 2] No such file or directory: 'Inputs/X.xlsx'

In [ ]:
# Data for reduced model
# MCR
# Data
X_reduced_mcr = X[['evaporation_and_natural_disperson','E_ss', 'E_sl', 'E_sw']]
y_m_b = label_binarize(y['MCR options'], classes=[10, 8, 2, -2])
n_classes = y_m_b.shape[1]
X_train_mcr, X_test_mcr, y_train_mcr, y_test_mcr = train_test_split(X_reduced_mcr, y_m_b, test_size=0.20, random_state=12)

# CDU
X_reduced_cdu = X[[ 'sufficient_mixing_energy','E_ssC', 'seawater']]
y_c_b = label_binarize(y['CDU options'], classes=[10, 8, -2, -10])
n_classes = y_c_b.shape[1]
X_train_cdu, X_test_cdu, y_train_cdu, y_test_cdu = train_test_split(X_reduced_cdu, y_c_b, test_size=0.20, random_state=12)

# ISB
X_reduced_isb = X[[ 'E_ssI','soot_pollution', 'displacement']]
y_i_b = label_binarize(y['ISB options'], classes=[10, 8, -2])
n_classes = y_i_b.shape[1]
X_train_isb, X_test_isb, y_train_isb, y_test_isb = train_test_split(X_reduced_isb, y_i_b, test_size=0.20, random_state=12)


In [ ]:
# Compare Algorithms
import pandas
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics

# prepare configuration for cross validation test harness
seed = 7
# prepare models
models = []
#models.append(('LR', LogisticRegression()))
models.append(('NB', OneVsRestClassifier(GaussianNB())))
# evaluate each model in turn
results = []
names = []
kfold = model_selection.KFold(n_splits=10)

for name, model in models:
	kfold = model_selection.KFold(n_splits=10)
	cv_results_m = model_selection.cross_val_score(model, X_reduced_mcr, y_m_b, cv=kfold, scoring='f1_samples') # , =scoring
	#cv_results2 = model_selection.cross_val_score(model, X_reduced_cdu, y_c_b, cv=kfold, scoring='roc_auc') # , =scoring
	results.append(cv_results_m)
	names.append(name)
    

cv_results_m = model_selection.cross_val_score(model, X, y_m_b, cv=kfold, scoring='roc_auc') # , =scoring
results.append(cv_results_m)
names.append('MCR_F')

cv_results_c = model_selection.cross_val_score(model, X, y_c_b, cv=kfold, scoring='roc_auc') # , =scoring
results.append(cv_results_c)
names.append('CDU_F')

cv_results_i = model_selection.cross_val_score(model, X, y_i_b, cv=kfold, scoring='roc_auc') # , =scoring
results.append(cv_results_i)
names.append('ISB_F')


cv_results_m2 = model_selection.cross_val_score(model, X_reduced_mcr, y_m_b, cv=kfold, scoring='roc_auc') # , =scoring
results.append(cv_results_m2)
names.append('MCR_r')

cv_results_c2 = model_selection.cross_val_score(model, X_reduced_cdu, y_c_b, cv=kfold, scoring='roc_auc') # , =scoring
results.append(cv_results_c2)
names.append('CDU_r')

cv_results_i2 = model_selection.cross_val_score(model, X_reduced_isb, y_i_b, cv=kfold, scoring='roc_auc') # , =scoring
results.append(cv_results_i2)
names.append('ISB_r')



In [ ]:
#sns.boxplot(x=results, y=names, palette="Set3")
boxplot_df = pd.DataFrame(results).T

fig4, ax = plt.subplots(figsize=(8,10))

sns.boxplot(data=boxplot_df, boxprops=dict(alpha=.3))
fig4.savefig('Outputs/boxplot_.png', dpi = 600)

boxplot_df.mean(axis=0)

### ROC Curve of reduced model

In [ ]:
y_train_mcr

#### MCR ROC curve

In [ ]:
classifier_mcr = OneVsRestClassifier(GaussianNB()).fit(X_train_mcr, y_train_mcr)
y_score_mcr = classifier_mcr.predict_proba(X_test_mcr)
y_pred_mcr = classifier_mcr.predict(X_test_mcr)

In [ ]:
# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
n_classes = y_test_mcr.shape[1]
for i in range(n_classes):
    fpr[i], tpr[i], _ = metrics.roc_curve(y_test_mcr[:, i], y_score_mcr[:, i])
    roc_auc[i] = metrics.auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = metrics.roc_curve(y_test_mcr.ravel(), y_score_mcr.ravel())
roc_auc["micro"] = metrics.auc(fpr["micro"], tpr["micro"])

# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = metrics.auc(fpr["macro"], tpr["macro"])

# Plot all ROC curves
fig1_mcrReduced = plt.figure()

colors = cycle(['limegreen', 'lightgreen', 'gray', 'lightgray'])
for i, color in zip(range(n_classes), colors):
    plt.plot(
        fpr[i],
        tpr[i],
        color=color,
        lw=1,
        label="ROC curve of class {0} (area = {1:0.2f})".format(i, roc_auc[i]),
    )

plt.plot([0, 1], [0, 1], "--", color='lightgray')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("MCR")
plt.legend(loc="lower right")
plt.show()


In [ ]:
# 
fig2_reduced = plt.figure()
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = metrics.roc_curve(y_test_mcr[:, i], y_score_mcr[:, i])
    roc_auc[i] = metrics.auc(fpr[i], tpr[i])
color_c = ['green', 'limegreen', 'blue', 'lightgray']
class_c = ['OK', 'Consider', 'Go next season','Unknown']
lw = [3,3,1,3]
#linestyle = ['solid', 'dashed', '-.', '-.']
# opacity/ transparency in lineplot ++

for i, color in zip(range(n_classes), color_c):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw[i], 
             label=class_c[i]) 
plt.plot([0, 1], [0, 1], '--', color= 'lightgray',lw=1)
plt.xlim([-0.05, 1.0])
plt.ylim([0.0, 1.05])

plt.ylabel('True Positive Rate')

plt.xlabel('False Positive Rate')
plt.legend(loc="lower right")

plt.show()
fig2_reduced.savefig('Outputs/ROC curve MCR 1 reduced.png', dpi = 600)

#### CDU ROC curve

In [ ]:
# CDU
classifier = OneVsRestClassifier(GaussianNB()).fit(X_train_cdu, y_train_cdu)
y_score_cdu = classifier.predict_proba(X_test_cdu)
y_pred_cdu = classifier.predict(X_test_cdu)
n_classes = y_test_cdu.shape[1]

fig2_reduced = plt.figure()
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = metrics.roc_curve(y_test_cdu[:, i], y_score_cdu[:, i])
    roc_auc[i] = metrics.auc(fpr[i], tpr[i])
color_c = ['green', 'blue', 'red', 'darkgray']
class_c = ['OK', 'Consider', 'Not recommended','Unknown']
linestyle = ['solid', 'dashed', 'solid', 'solid']
lw = [3,3,1,3]
# opacity/ transparency in lineplot ++

for i, color in zip(range(n_classes), color_c):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw[i], linestyle=linestyle[i],
             label=class_c[i]) 
plt.plot([0, 1], [0, 1], '--', color= 'lightgray',lw=1)
plt.xlim([-0.05, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc="lower right")

plt.show()
fig2_reduced.savefig('Outputs/ROC curve cdu 1 reduced.png', dpi = 600)

for i in range(n_classes):
    fpr[i], tpr[i], _ = metrics.roc_curve(y_test_cdu[:, i], y_score_cdu[:, i])
    roc_auc[i] = metrics.auc(fpr[i], tpr[i])
roc_auc

#### ISB ROC Curve

In [ ]:
# ISB
classifier = OneVsRestClassifier(GaussianNB()).fit(X_train_isb, y_train_isb)
y_score_isb = classifier.predict_proba(X_test_isb)
y_pred_isb = classifier.predict(X_test_isb)
n_classes = y_test_isb.shape[1]

fig4 = plt.figure()
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(y_test_isb.shape[1]):
    fpr[i], tpr[i], _ = metrics.roc_curve(y_test_isb[:, i], y_score_isb[:, i])
    roc_auc[i] = metrics.auc(fpr[i], tpr[i])
color_i = ['green', 'limegreen',  'darkgray']
class_i = ['OK', 'Consider', 'Unknown']
linestyle = ['solid', 'solid', 'solid']

for i, color in zip(range(y_test_isb.shape[1]), color_i):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw[i], 
             label=class_i[i], linestyle=linestyle[i]) 
plt.plot([0, 1], [0, 1], '--', lw=1, color='lightgray')
plt.xlim([-0.05, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc="lower right")

plt.show()
fig4.savefig('Outputs/ROC curve isb, reduced, name.png', dpi = 600)

# print roc values
for i in range(n_classes):
    fpr[i], tpr[i], _ = metrics.roc_curve(y_test_isb[:, i], y_score_isb[:, i])
    roc_auc[i] = metrics.auc(fpr[i], tpr[i])
roc_auc

In [ ]:
"""1. ROC curve in this Notebook is for OnevsRest classifier (not Multioutput classifier)

"""